In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
sun_data = pd.read_csv("train.csv")
sun_data.drop(['Day', 'Hour','Minute'], axis='columns', inplace=True)
sun_data.head()

,DHI,DNI,WS,RH,T,TARGET
0,0,0,1.5,69.08,-12,0.0
1,0,0,1.5,69.06,-12,0.0
2,0,0,1.6,71.78,-12,0.0
3,0,0,1.6,71.75,-12,0.0
4,0,0,1.6,75.20,-12,0.0


In [3]:
sun_data.tail()

,DHI,DNI,WS,RH,T,TARGET
52555,0,0,2.4,70.70,-4,0.0
52556,0,0,2.4,66.79,-4,0.0
52557,0,0,2.2,66.78,-4,0.0
52558,0,0,2.1,67.72,-4,0.0
52559,0,0,2.1,67.70,-4,0.0


In [4]:
print(1092/7)
print(48*7)

156.0
336


In [5]:
def create_data(data):
    data_X, data_Y = [], []
#     input_data_X, input_data_Y = [], []
    data_X = np.array(sun_data.loc[:, ["DHI", "DNI", "WS", "RH", "T"]].values)
    data_Y = np.array(sun_data.loc[:, ["TARGET"]].values)
#     for i in range(1092//7):
#         input_data_X += data_X[i*(48*7) : (i+1)*(48*7)]
#         input_data_Y += data_Y[i*(48*7) : (i+1)*(48*7)]
    return data_X, data_Y
#     return input_data_X, input_data_Y

X, Y = create_data(sun_data)
X[:10], Y[:10]

(array([[  0.  ,   0.  ,   1.5 ,  69.08, -12.  ],
        [  0.  ,   0.  ,   1.5 ,  69.06, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  71.78, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  71.75, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  75.2 , -12.  ],
        [  0.  ,   0.  ,   1.5 ,  69.29, -11.  ],
        [  0.  ,   0.  ,   1.5 ,  72.56, -11.  ],
        [  0.  ,   0.  ,   1.4 ,  72.55, -11.  ],
        [  0.  ,   0.  ,   1.3 ,  74.62, -11.  ],
        [  0.  ,   0.  ,   1.3 ,  74.61, -11.  ]]), array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

In [6]:
print(len(X), len(Y))
print(len(X[0]), len(Y[0]))
X.shape, Y.shape

52560 52560
5 1


((52560, 5), (52560, 1))

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scaler = StandardScaler()
# input_scaler = scaler.fit(X)
# output_scaler = scaler.fit(Y)
# X = input_scaler.transform(X)
# Y = output_scaler.transform(Y)

scaler = MinMaxScaler(feature_range = (0, 1))
input_scaler = scaler.fit(X)
output_scaler = scaler.fit(Y)
X = input_scaler.transform(X)
Y = output_scaler.transform(Y)

# X = X.reshape(-1,336,5)
# Y = Y.reshape(-1,7,1)
X[0], Y[0]

(array([ 0.        ,  0.        ,  0.01501292,  0.69139502, -0.12010336]),
 array([0.]))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((39420, 5), (39420, 1), (13140, 5), (13140, 1))

In [10]:
def create_dataset (X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

In [11]:
X_train, y_train = create_dataset(X_train, y_train, 48)
X_test, y_test= create_dataset(X_test, y_test, 48)

In [12]:
print('X_train.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_train.shape)

X_train.shape:  (13092, 48, 5)
y_train.shape:  (39372, 1)
X_test.shape:  (13092, 48, 5)
y_test.shape:  (39372, 1)


In [ ]:
#model_1
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Activation, LSTM, Dropout, Bidirectional, GRU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(64, dropout=0.3, activation="elu", return_sequences=True, input_shape=[48,5]))
model.add(LSTM(64, dropout=0.3, activation="elu"))
model.add(Dense(1))
optimizer = RMSprop(0.001)
model.compile(loss="mean_squared_error", optimizer = optimizer)

# filepath = "weights/" + "lstm_model" + "-{epoch:02d}-{val_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]  /// callbacks=callbacks_list

history = model.fit(X_train, y_train, epochs=100, batch_size=128, shuffle=False, validation_split=0.1)

Epoch 1/100
277/277 [==============================] - 46s 159ms/step - loss: 0.0742 - val_loss: 0.0672
Epoch 2/100
277/277 [==============================] - 43s 156ms/step - loss: 0.0664 - val_loss: 0.0674
Epoch 3/100
277/277 [==============================] - 44s 159ms/step - loss: 0.0662 - val_loss: 0.0670
Epoch 4/100
277/277 [==============================] - 43s 154ms/step - loss: 0.0661 - val_loss: 0.0671
Epoch 5/100
277/277 [==============================] - 44s 159ms/step - loss: 0.0660 - val_loss: 0.0670
Epoch 6/100
277/277 [==============================] - 44s 159ms/step - loss: 0.0660 - val_loss: 0.0670
Epoch 7/100
277/277 [==============================] - 45s 161ms/step - loss: 0.0660 - val_loss: 0.0670
Epoch 8/100
277/277 [==============================] - 43s 156ms/step - loss: 0.0660 - val_loss: 0.0670
Epoch 9/100
277/277 [==============================] - 44s 158ms/step - loss: 0.0660 - val_loss: 0.0669
Epoch 10/100
277/277 [==============================] - 44s 160m

In [ ]:
#model_2
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, Bidirectional, GRU

# Create BiLSTM model
def create_model_bilstm(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(units = units)))
    model.add(Dense(1))
    #Compile model
    model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])
    return model

# Create LSTM or GRU model
def create_model(units, m):
    model = Sequential()
    model.add(m (units = units, return_sequences = True,
                input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2))
    model.add(m (units = units))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))

    #Compile model
    model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])
    return model

# BiLSTM
model_bilstm = create_model_bilstm(64)

# GRU and LSTM
model_gru = create_model(64, GRU)
model_lstm = create_model(64, LSTM)

In [ ]:
#model_2_fit
def fit_model(model):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
    history = model.fit(X_train, y_train, epochs = 100, validation_split = 0.2, batch_size = 64,
                        shuffle = False, callbacks = [early_stop])
    return history
history_bilstm = fit_model(model_bilstm)
history_lstm = fit_model(model_lstm)
history_gru = fit_model(model_gru)

In [ ]:
def plot_loss (history):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
plot_loss (history)
